In [4]:
import numpy as np
import pandas as pd
from pathlib import Path
import os
import os
import yaml
os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'
import torch
import numpy as np
from os.path import join

from helpers.predict_utils import Config, get_data
from helpers.models import TimeSeriesRegressorWrapper
from paper_utils.utils import participants

allow_tf32 = True
if allow_tf32:
    torch.backends.cuda.matmul.allow_tf32 = True

In [ ]:
online_participants = [part for part in participants if 'P6_' not in part]

config_name = 'modular_online'
sampling_frequency = 60    
device = torch.device("cpu")

for person_dir in online_participants:
    print(f'Processing {person_dir}...')
    with open(join('data', person_dir, 'configs', f'{config_name}.yaml'), 'r') as file:
        wandb_config = yaml.safe_load(file)
        config = Config(wandb_config)
    
    data_dirs = [join('data', person_dir, 'recordings', recording, 'experiments', '1') for recording in config.recordings]
    test_dirs = [join('data', person_dir, 'recordings', recording, 'experiments', '1') for recording in config.test_recordings] if config.test_recordings is not None else []
    
    intact_hand = config.targets[0][0]
    config.person_dir = person_dir
    config.intact_hand = intact_hand

    for perturb in [True, False]:
        print(f'\tPerturbation: {perturb}')
        experiment_name = f'non_' if not perturb else '' + 'perturb'
        perturb_file = join('data', person_dir, 'online_trials', experiment_name, 'perturber.npy')
        trainsets, valsets, combined_sets, testsets = get_data(config, data_dirs, intact_hand, test_dirs=test_dirs, perturb_file=perturb_file)
        
        model = TimeSeriesRegressorWrapper(device=device, input_size=len(config.features), output_size=len(config.targets), **config.to_dict())
        for condition in ['before', 'after']:
            print(f'\t\tCondition: {condition}')

            model.to('cpu')
            if condition == 'before':
                model.load(join('data', person_dir, 'online_trials', experiment_name, 'models', f'{person_dir}-online_0.pt'))
            else:
                model.load(join('data', person_dir, 'online_trials', experiment_name, 'models', f'{person_dir}-online_last.pt'))

            model.to(device)
            model.eval()
            
            for recording, testset in zip(config.test_recordings, testsets):
                pred = model.predict(testset, config.features, config.targets).squeeze(0).to("cpu").detach().numpy()
                save_dir = join('data', person_dir, 'recordings', recording, 'experiments', '1')
                testset[config.targets] = pred
                testset.to_parquet(save_dir, f"pred_angles-{config.name}_{condition}Online_{experiment_name}.parquet")
                
            for recording, valset in zip(config.recordings, valsets):
                pred = model.predict(valset, config.features, config.targets).squeeze(0).to("cpu").detach().numpy()
                save_dir = join('data', person_dir, 'recordings', recording, 'experiments', '1')
                valset[config.targets] = pred
                valset.to_parquet(save_dir, f"pred_angles-{config.name}_{condition}Online_{experiment_name}.parquet")                 

Processing P_149...
	Perturbation: True
{'name': 'P_149', 'recordings': ['thumbFlEx', 'thumbAbAd', 'indexFlEx', 'mrpFlEx', 'fingersFlEx', 'wristFlEx', 'handOpCl', 'pinchOpCl', 'pointOpCl'], 'test_recordings': ['keyOpCl', 'wristFlHandCl', 'indexFlDigitsEx'], 'features': [('emg', '0'), ('emg', '1'), ('emg', '2'), ('emg', '4'), ('emg', '10'), ('emg', '11'), ('emg', '12'), ('emg', '13')], 'early_stopping_patience': 1000, 'early_stopping_delta': 0.05, 'learning_rate': 0.005, 'n_epochs': 1000, 'seq_len': 64, 'targets': [('Left', 'indexAng'), ('Left', 'midAng'), ('Left', 'ringAng'), ('Left', 'pinkyAng'), ('Left', 'thumbInPlaneAng'), ('Left', 'thumbOutPlaneAng'), ('Left', 'wristFlex')], 'warmup_steps': 0, 'batch_size': 8, 'wandb_mode': 'offline', 'wandb_project': 'study_participants', 'model_type': 'ModularModel', 'activation_model': {'model_type': 'DenseNet', 'hidden_size': 64, 'n_layers': 3, 'n_freeze_epochs': 0}, 'muscle_model': {'model_type': 'PhysMuscleModel', 'n_freeze_epochs': 1}, 'join

FileNotFoundError: [Errno 2] No such file or directory: 'data/P_149/online_trials/perturb/models/P_149-online_0.pt'